In [1]:
import reverse_geocode
import pandas as pd
from geopy.geocoders import Nominatim
import shapely
import shapely.wkt
from shapely.geometry import Point, Polygon
import overpy

In [2]:
coordinates =(40.44850748776656, -80.01171877053345), (-37.81, 144.96), (31.76, 35.21)
result = reverse_geocode.search(coordinates) 
result
# result is a list containing ordered dictionary. 
# pprint.pprint(result)

[{'country_code': 'US', 'city': 'Pittsburgh', 'country': 'United States'},
 {'country_code': 'AU', 'city': 'Melbourne', 'country': 'Australia'},
 {'country_code': 'IL', 'city': 'Jerusalem', 'country': 'Israel'}]

In [3]:
locator = Nominatim(user_agent="myGeocoder")
coordinates = "40.44850748776656, -80.01171877053345"
location = locator.reverse(coordinates)
location.raw

{'place_id': 193210505,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 459489069,
 'lat': '40.44833377332564',
 'lon': '-80.01172306512626',
 'display_name': 'Reedsdale Street, North Shore, Pittsburgh, Allegheny County, Pennsylvania, 14233, United States',
 'address': {'road': 'Reedsdale Street',
  'neighbourhood': 'North Shore',
  'city': 'Pittsburgh',
  'county': 'Allegheny County',
  'state': 'Pennsylvania',
  'postcode': '14233',
  'country': 'United States',
  'country_code': 'us'},
 'boundingbox': ['40.4483317', '40.4483664', '-80.0130428', '-80.0116392']}

In [8]:
location = locator.geocode("Champ de Mars, Paris, France")
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

Latitude = 48.85614465, Longitude = 2.297820393322227


## Making OverPass API Queries

In [2]:
pgh_cells = pd.read_csv('../Data/PGH/DemandPrediction/pgh_grid_fit.csv')
pgh_cells = pgh_cells.drop(columns=['Unnamed: 0'])
pgh_cells.head(2)

,cell_num,geometry,centroid,stations,capacity,tractce10,total_households,<$10K,$10K-15K,$15K-25K,...,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31
0,51,"POLYGON ((-80.0953367046031 40.44883540123336,...",POINT (-80.09304131033451 40.45113079550194),0,0,563000,1306,59,33,79,...,0,0,0,0,0,0,0,0,0,0
1,52,"POLYGON ((-80.0953367046031 40.45342618977051,...",POINT (-80.09304131033451 40.45572158403908),0,0,563000,1306,59,33,79,...,0,0,0,0,0,0,0,0,0,0


In [4]:
for row,col in pgh_cells.iterrows():
    pgh_cells.at[row, 'geometry'] = shapely.wkt.loads(pgh_cells.loc[row]['geometry'])
    
    if pgh_cells.loc[row]['stations'] > 0:
        print(row)
        print("stations " + str(pgh_cells.loc[row]['stations']))
        print("geometry: " + str(pgh_cells.loc[row]['geometry'].bounds))

203
stations 1
geometry: (-80.00352093385995, 40.43047224708476, -79.99893014532279, 40.43506303562191)
224
stations 2
geometry: (-80.01270251093426, 40.44424461269621, -80.0081117223971, 40.44883540123336)
225
stations 1
geometry: (-80.0081117223971, 40.44424461269621, -80.00352093385995, 40.44883540123336)
226
stations 1
geometry: (-80.01729329947142, 40.44883540123336, -80.01270251093426, 40.45342618977051)
231
stations 1
geometry: (-80.01270251093426, 40.45342618977051, -80.0081117223971, 40.45801697830766)
232
stations 1
geometry: (-80.0081117223971, 40.45342618977051, -80.00352093385995, 40.45801697830766)
250
stations 2
geometry: (-80.0081117223971, 40.43965382415906, -80.00352093385995, 40.44424461269621)
251
stations 3
geometry: (-80.00352093385995, 40.43506303562191, -79.99893014532279, 40.43965382415906)
252
stations 3
geometry: (-80.00352093385995, 40.43965382415906, -79.99893014532279, 40.44424461269621)
253
stations 3
geometry: (-79.99893014532279, 40.43506303562191, -79.

In [13]:
print(pgh_cells.loc[0]['geometry'].bounds[0]) #x1
print(pgh_cells.loc[0]['geometry'].bounds[1]) #y1
print(pgh_cells.loc[0]['geometry'].bounds[2]) #x2
print(pgh_cells.loc[0]['geometry'].bounds[3]) #y2

-80.0953367046031
40.44883540123336
-80.09074591606594
40.45342618977051


In [87]:
nodesresult = []

In [ ]:
api = overpy.Overpass()
cells_poi = []

for row, col in pgh_cells.iterrows():
    if row < 800 and row > 0:
        querystr = 'node('+str(pgh_cells.loc[row]['geometry'].bounds[1]) + ',' + str(pgh_cells.loc[row]['geometry'].bounds[0]) +',' + str(pgh_cells.loc[row]['geometry'].bounds[3]) + ',' + str(pgh_cells.loc[row]['geometry'].bounds[2]) + '); out;'
        result = api.query(querystr)
        nodes = result.nodes #list of all nodes that returned from query
        nodesresult.append(nodes)

In [92]:
len(nodesresult)

727

In [98]:
count = 0

node_cells_dict = {}

for nodes in nodesresult:
    print("NODE LIST = LOC IN CELL DF: " + str(count))
    node_cells_dict[count] = []
    if len(nodes) > 0:
        for node in nodes:
            if len(node.tags) > 0:
                if 'power' not in node.tags.keys() and 'barrier' not in node.tags.keys():
                    print(node.tags)
                    node_cells_dict[count].append(node.tags)
                
    count += 1

NODE LIST = LOC IN CELL DF: 0
{'railway': 'buffer_stop', 'source': 'bing'}
{'railway': 'buffer_stop', 'source': 'bing'}
NODE LIST = LOC IN CELL DF: 1
NODE LIST = LOC IN CELL DF: 2
{'highway': 'stop', 'source': 'bing'}
{'railway': 'level_crossing'}
{'railway': 'level_crossing'}
{'highway': 'turning_circle', 'source': 'bing'}
NODE LIST = LOC IN CELL DF: 3
{'man_made': 'storage_tank', 'source': 'bing'}
NODE LIST = LOC IN CELL DF: 4
{'abandoned:railway': 'station', 'addr:state': 'PA', 'ele': '242', 'gnis:county_name': 'Allegheny', 'gnis:feature_id': '1196592', 'gnis:import_uuid': '57871b70-0100-4405-bb30-88b2e001a944', 'gnis:reviewed': 'no', 'name': 'Superior Station', 'source': 'USGS Geonames'}
{'railway': 'buffer_stop', 'source': 'bing'}
NODE LIST = LOC IN CELL DF: 5
{'highway': 'turning_circle'}
NODE LIST = LOC IN CELL DF: 6
{'ele': '235', 'gnis:Class': 'Populated Place', 'gnis:County': 'Allegheny', 'gnis:County_num': '003', 'gnis:ST_alpha': 'PA', 'gnis:ST_num': '42', 'gnis:id': '120281

In [99]:
node_cells_dict

{0: [{'railway': 'buffer_stop', 'source': 'bing'},
  {'railway': 'buffer_stop', 'source': 'bing'}],
 1: [],
 2: [{'highway': 'stop', 'source': 'bing'},
  {'railway': 'level_crossing'},
  {'railway': 'level_crossing'},
  {'highway': 'turning_circle', 'source': 'bing'}],
 3: [{'man_made': 'storage_tank', 'source': 'bing'}],
 4: [{'abandoned:railway': 'station',
   'addr:state': 'PA',
   'ele': '242',
   'gnis:county_name': 'Allegheny',
   'gnis:feature_id': '1196592',
   'gnis:import_uuid': '57871b70-0100-4405-bb30-88b2e001a944',
   'gnis:reviewed': 'no',
   'name': 'Superior Station',
   'source': 'USGS Geonames'},
  {'railway': 'buffer_stop', 'source': 'bing'}],
 5: [{'highway': 'turning_circle'}],
 6: [{'ele': '235',
   'gnis:Class': 'Populated Place',
   'gnis:County': 'Allegheny',
   'gnis:County_num': '003',
   'gnis:ST_alpha': 'PA',
   'gnis:ST_num': '42',
   'gnis:id': '1202817',
   'import_uuid': 'bb7269ee-502a-5391-8056-e3ce0e66489c',
   'is_in': 'Allegheny,Pennsylvania,Pa.,PA,

### Collecting OverPass API Query Data

In [80]:
(-79.99893014532279, 40.43506303562191, -79.99433935678563, 40.43965382415906)
result = api.query("""node(40.43506303562191,-79.99893014532279,40.43965382415906,-79.99433935678563); out;"""
                  )

In [81]:
len(result.nodes)

1622

In [82]:
node = result.nodes[3]

In [83]:
node.tags

{'highway': 'traffic_signals'}

In [84]:
for node in result.nodes:
    if len(node.tags) > 0:
        print(node.tags)

{'highway': 'traffic_signals'}
{'highway': 'traffic_signals'}
{'highway': 'traffic_signals'}
{'highway': 'traffic_signals'}
{'highway': 'traffic_signals'}
{'noexit': 'yes'}
{'highway': 'traffic_signals'}
{'highway': 'traffic_signals'}
{'highway': 'traffic_signals'}
{'highway': 'traffic_signals'}
{'highway': 'traffic_signals', 'traffic_signals': 'signal'}
{'light_rail': 'yes', 'name': 'First Avenue', 'network': 'PAT', 'operator': 'Port Authority of Allegheny County', 'public_transport': 'stop_position', 'railway': 'stop', 'wheelchair': 'yes'}
{'addr:state': 'PA', 'ele': '231', 'gnis:county_name': 'Allegheny', 'gnis:feature_id': '2430678', 'gnis:import_uuid': '57871b70-0100-4405-bb30-88b2e001a944', 'gnis:reviewed': 'no', 'name': 'Renaissance III', 'source': 'USGS Geonames'}
{'addr:state': 'PA', 'ele': '233', 'gnis:county_name': 'Allegheny', 'gnis:feature_id': '2430643', 'gnis:import_uuid': '57871b70-0100-4405-bb30-88b2e001a944', 'gnis:reviewed': 'no', 'name': 'Pittsburgh Parks and Recrea